# Projet DATA Job

## Deuxième itération des modèles machines learning pour le projet Data Job

Auteurs : Monya .M et Nejib B.

### Introduction : 

Lors de la première itération des modèles de machine learning, nous avons observés que deux modèles se démarquaient sur nos jeux d'entraînements et de tests. Il s'agit des modèles : **Decision Tree** (score : *78%*) et **Random Forest** (score : *80%*).  

Néanmoins, certaines classes de nos jeux d'entraînements et de tests n'ont pas été correctement classifiées. Cette seconde itération doit nous aider à affiner la sélection des données des jeux d'entraînements et de tests.  

Nous avons fait le choix pour cette deuxième itération de ne pas utiliser les colonnes contenant des valeurs numériques afin d'utiliser les valeurs textuelles de nos colonnes. Nos deux modèles de machine learning sont capables de classifier les données textuelles. Nous espérons ainsi, obtenir un meilleur résultat.

### Importation du nouveau jeu de donnée : 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_it2.csv', sep = ';')

#Affichage des 5 premières lignes du jeu de données et de sa taille
display(df.head())
df.shape

,diploma,position,programming_python,programming_r,programming_sql,programming_c,programming_cplus,programming_java,programming_javascript,programming_julia,...,share_deploy_data_analysis_ml_streamlit,share_deploy_data_analysis_ml_nbviewer,share_deploy_data_analysis_ml_github,share_deploy_data_analysis_ml_personal_blog,share_deploy_data_analysis_ml_kaggle,share_deploy_data_analysis_ml_colab,share_deploy_data_analysis_ml_shiny,share_deploy_data_analysis_ml_dont_share,share_deploy_data_analysis_ml_other,primary_tool_to_analyze_data_text_response
0,Master’s degree,Data Engineer,Python,R,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Business intelligence software (Salesforce, Ta..."
1,Master’s degree,Data Scientist,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,"Local development environments (RStudio, Jupyt..."
2,Bachelor’s degree,Data Analyst,Python,R,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Doctoral degree,Data Scientist,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Cloud-based data software & APIs (AWS, GCP, Az..."
4,Bachelor’s degree,Data Engineer,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, G..."


(5670, 200)

Ici, il n'est plus question d'utiliser seulement 18 colonnes de notre jeu de donnée, mais bel et bien d'essayer d'utiliser le plus de colonnes possibles.  

Les colonnes contenants des valeurs numériques et les colonnes concernant les questions sur les technologies ou sujets que souhaitent approfondir les répondants dans les deux prochaines années ont été supprimées.  

En effet, ces colonnes ne semblaient que peu ou pas pertinentes pour cette nouvelle itération. 

### Première observation du jeu de donnée : 

In [2]:
#Afficher les informations de df :
display(df.info())

#Lister les colonnes de df : 
display(df.columns)

#Afficher la répartition des valeurs manquantes dans df : 
display(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5670 entries, 0 to 5669
Columns: 200 entries, diploma to primary_tool_to_analyze_data_text_response
dtypes: object(200)
memory usage: 8.7+ MB


None

Index(['diploma', 'position', 'programming_python', 'programming_r',
       'programming_sql', 'programming_c', 'programming_cplus',
       'programming_java', 'programming_javascript', 'programming_julia',
       ...
       'share_deploy_data_analysis_ml_streamlit',
       'share_deploy_data_analysis_ml_nbviewer',
       'share_deploy_data_analysis_ml_github',
       'share_deploy_data_analysis_ml_personal_blog',
       'share_deploy_data_analysis_ml_kaggle',
       'share_deploy_data_analysis_ml_colab',
       'share_deploy_data_analysis_ml_shiny',
       'share_deploy_data_analysis_ml_dont_share',
       'share_deploy_data_analysis_ml_other',
       'primary_tool_to_analyze_data_text_response'],
      dtype='object', length=200)

diploma                                          0
position                                         0
programming_python                             706
programming_r                                 4030
programming_sql                               2744
                                              ... 
share_deploy_data_analysis_ml_colab           4959
share_deploy_data_analysis_ml_shiny           5456
share_deploy_data_analysis_ml_dont_share      4615
share_deploy_data_analysis_ml_other           5561
primary_tool_to_analyze_data_text_response    1227
Length: 200, dtype: int64

On constate tout de suite, que les colonnes de notre jeu de donnée contiennent énormément de valeurs manquantes. Cela est cohérent avec le type de donnée que nous avons à notre disposition, à savoir les réponses d'un questionnaire à choix multiples.  

Les valeurs manquantes représentent une non-réponse de la part du répondant. Elles seront ici remplacées par : **"no_reply"**

### Informations essentielles 

In [3]:
#Fonction qui permet l'affichage des informations suivantes : type, % des valeurs manquantes, nbre de valeurs uniques, valeur unique, valeur moyenne ou valeur la plus fréquente
def summary(df):
    table = pd.DataFrame(
        index=df.columns,
        columns=['type_info', '%_missing_values', 'nb_unique_values'])
    table.loc[:, 'type_info'] = df.dtypes.values
    table.loc[:, '%_missing_values'] = (df.isna().sum().values / len(df))*100
    table.loc[:, 'nb_unique_values'] = df.nunique().values

    def get_list_unique_values(df):
        dict_ = {}
        for col in df.columns:
            if df[col].nunique() < 6:
                dict_[col] = list(df[col].unique())
            else:
                if df[col].dtypes == "O":
                    dict_[col] = "Too much categories..."
                else:
                    dict_[col] = "Too much values..."
        return pd.DataFrame({'unique_values': dict_.values()},
                            index=df.columns)

    infos_table_int1 = pd.merge(table,
                                get_list_unique_values(df),
                                left_index=True,
                                right_index=True)

    def get_mean_mode(df):
        dict_ = {}
        for col in df.columns:
            if df[col].dtypes == "O":
                dict_[col] = df[col].mode()[0]
            else:
                dict_[col] = df[col].mean()
        return pd.DataFrame(pd.Series(dict_), columns=["mean_mode_values"])

    infos_table_int2 = pd.merge(infos_table_int1,
                                get_mean_mode(df),
                                left_index=True,
                                right_index=True)

    def alerts(df):
        thresh_na = 0.25
        thresh_balance = 0.8
        dict_ = {}
        for col in df.columns:
            if (df[col].count() / len(df)) < thresh_na:
                dict_[col] = "Too much missing values ! "
            elif df[col].value_counts(
                    normalize=True).values[0] > thresh_balance:
                dict_[col] = "It's imbalanced !"
            else:
                dict_[col] = "Nothing to report"

        return pd.DataFrame(pd.Series(dict_), columns=["flag"])

    infos_table = pd.merge(infos_table_int2,
                           alerts(df),
                           left_index=True,
                           right_index=True)

    return infos_table

pd.set_option('display.max_rows', None)
summary(df)

,type_info,%_missing_values,nb_unique_values,unique_values,mean_mode_values,flag
diploma,object,0.000000,7,Too much categories...,Master’s degree,Nothing to report
position,object,0.000000,4,"[Data Engineer, Data Scientist, Data Analyst, ...",Data Scientist,Nothing to report
programming_python,object,12.451499,1,"[Python, nan]",Python,It's imbalanced !
programming_r,object,71.075838,1,"[R, nan]",R,It's imbalanced !
programming_sql,object,48.395062,1,"[SQL, nan]",SQL,It's imbalanced !
programming_c,object,89.276896,1,"[nan, C]",C,Too much missing values !
programming_cplus,object,86.313933,1,"[nan, C++]",C++,Too much missing values !
programming_java,object,87.848325,1,"[nan, Java]",Java,Too much missing values !
programming_javascript,object,87.777778,1,"[nan, Javascript]",Javascript,Too much missing values !
programming_julia,object,98.183422,1,"[nan, Julia]",Julia,Too much missing values !


## Preprocessing : 

In [4]:
#Creation d'un nouveau dataframe pour réaliser le preprocessing : 

df_prep = df.copy()

#Affichage du nouveau dataframe :
df_prep.head()


,diploma,position,programming_python,programming_r,programming_sql,programming_c,programming_cplus,programming_java,programming_javascript,programming_julia,...,share_deploy_data_analysis_ml_streamlit,share_deploy_data_analysis_ml_nbviewer,share_deploy_data_analysis_ml_github,share_deploy_data_analysis_ml_personal_blog,share_deploy_data_analysis_ml_kaggle,share_deploy_data_analysis_ml_colab,share_deploy_data_analysis_ml_shiny,share_deploy_data_analysis_ml_dont_share,share_deploy_data_analysis_ml_other,primary_tool_to_analyze_data_text_response
0,Master’s degree,Data Engineer,Python,R,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Business intelligence software (Salesforce, Ta..."
1,Master’s degree,Data Scientist,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,"Local development environments (RStudio, Jupyt..."
2,Bachelor’s degree,Data Analyst,Python,R,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Doctoral degree,Data Scientist,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Cloud-based data software & APIs (AWS, GCP, Az..."
4,Bachelor’s degree,Data Engineer,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, G..."


### Colonne diploma : 

In [5]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
diploma_dic = {'diploma' : {'Master’s degree' : 'master', 'Bachelor’s degree' : 'bachelor', 
                            'Doctoral degree' : 'doctorate',
                            'Some college/university study without earning a bachelor’s degree':'no_superior_degree',
                            'Professional degree' : 'professional_degree', 
                            'I prefer not to answer' : 'no_information',
                            'No formal education past high school':'no_superior_degree'}}

In [6]:
#Mise à jour des valeurs de la colonne 'diploma'
df_prep.replace(diploma_dic, inplace = True)

#Vérification que tout s'affiche correctement.
display(df_prep.head())

#Affichage des nouvelles valeurs uniques de la colonne diploma :
print('Les nouvelles valeurs uniques sont :',df_prep.diploma.unique())

,diploma,position,programming_python,programming_r,programming_sql,programming_c,programming_cplus,programming_java,programming_javascript,programming_julia,...,share_deploy_data_analysis_ml_streamlit,share_deploy_data_analysis_ml_nbviewer,share_deploy_data_analysis_ml_github,share_deploy_data_analysis_ml_personal_blog,share_deploy_data_analysis_ml_kaggle,share_deploy_data_analysis_ml_colab,share_deploy_data_analysis_ml_shiny,share_deploy_data_analysis_ml_dont_share,share_deploy_data_analysis_ml_other,primary_tool_to_analyze_data_text_response
0,master,Data Engineer,Python,R,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Business intelligence software (Salesforce, Ta..."
1,master,Data Scientist,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,"Local development environments (RStudio, Jupyt..."
2,bachelor,Data Analyst,Python,R,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,doctorate,Data Scientist,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Cloud-based data software & APIs (AWS, GCP, Az..."
4,bachelor,Data Engineer,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, G..."


Les nouvelles valeurs uniques sont : ['master' 'bachelor' 'doctorate' 'no_superior_degree'
 'professional_degree' 'no_information']


### Colonne position : 

In [7]:
#Création d'une nouvelle variable dat : 
target = df_prep.position.copy()

#Affichage des informations : 
display(target.head())
display(target.shape)

0     Data Engineer
1    Data Scientist
2      Data Analyst
3    Data Scientist
4     Data Engineer
Name: position, dtype: object

(5670,)

Pour éviter toutes futures erreurs nous allons retirer directement la colonne position de df_prep.

In [8]:
#Suppression de la colonne position de df_prep : 
df_prep = df_prep.drop(['position'], axis = 1)
df_prep.head()

,diploma,programming_python,programming_r,programming_sql,programming_c,programming_cplus,programming_java,programming_javascript,programming_julia,programming_swift,...,share_deploy_data_analysis_ml_streamlit,share_deploy_data_analysis_ml_nbviewer,share_deploy_data_analysis_ml_github,share_deploy_data_analysis_ml_personal_blog,share_deploy_data_analysis_ml_kaggle,share_deploy_data_analysis_ml_colab,share_deploy_data_analysis_ml_shiny,share_deploy_data_analysis_ml_dont_share,share_deploy_data_analysis_ml_other,primary_tool_to_analyze_data_text_response
0,master,Python,R,SQL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Business intelligence software (Salesforce, Ta..."
1,master,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,"Local development environments (RStudio, Jupyt..."
2,bachelor,Python,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,doctorate,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,NaN,NaN,NaN,NaN,NaN,"Cloud-based data software & APIs (AWS, GCP, Az..."
4,bachelor,Python,NaN,SQL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, G..."


### Gestion des valeurs manquantes : 

Notre jeu de données comporte de nombreuses valeurs manquantes, nous avons fait le choix pour cette seconde itération de remplacer les valeurs manquantes : "NA" par "no_reply". S'agissant des réponses à un questionnaire, les lignes qui comportent des "NA" correspondent aux questions auxquels les répondants n'ont pas répondu.

In [9]:
#Remplacer l'ensemble des NA par 'no_reply' :
df_prep = df_prep.fillna('no_reply')

#Verification :
display(df_prep.isna().sum())

diploma                                                                                0
programming_python                                                                     0
programming_r                                                                          0
programming_sql                                                                        0
programming_c                                                                          0
programming_cplus                                                                      0
programming_java                                                                       0
programming_javascript                                                                 0
programming_julia                                                                      0
programming_swift                                                                      0
programming_bash                                                                       0
programming_matlab   

### Colonne use_ide_jupyter :

In [10]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_ide_jupyter_dic = {'use_ide_jupyter' : {'Jupyter (JupyterLab, Jupyter Notebooks, etc) ' : 'Jupyter',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_ide_jupyter_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_ide_jupyter.unique())

Les valeurs uniques sont : ['no_reply' 'Jupyter']


### Colonne use_ide_vim_emacs :

In [11]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_ide_vim_emacs_dic = {'use_ide_vim_emacs' : {'Vim / Emacs' : 'Vim_Emacs',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_ide_vim_emacs_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_ide_vim_emacs.unique())

Les valeurs uniques sont : ['no_reply' 'Vim_Emacs']


### Colonne use_data_visualization_plotly :

In [12]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_data_visualization_plotly_dic = {'use_data_visualization_plotly' : {'Plotly / Plotly Express' : 'Plotly_Plotly_Express',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_data_visualization_plotly_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_data_visualization_plotly.unique())

Les valeurs uniques sont : ['no_reply' 'Plotly_Plotly_Express']


### Colonne use_data_visualization_ggplot :

In [13]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_data_visualization_ggplot_dic = {'use_data_visualization_ggplot' : {'Ggplot / ggplot2' : 'Ggplot_ggplot2',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_data_visualization_ggplot_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_data_visualization_ggplot.unique())

Les valeurs uniques sont : ['Ggplot_ggplot2' 'no_reply']


### Colonne use_data_visualization_leaflet_folium :

In [14]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_data_visualization_leaflet_folium_dic = {'use_data_visualization_leaflet_folium' : {'Leaflet / Folium' : 'Leaflet_Folium',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_data_visualization_leaflet_folium_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_data_visualization_leaflet_folium.unique())

Les valeurs uniques sont : ['no_reply' 'Leaflet_Folium']


### Colonne use_machine_learning_fastai :

In [15]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_machine_learning_fastai_dic = {'use_machine_learning_fastai' : {'Fast.ai' : 'Fast_ai',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_machine_learning_fastai_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_machine_learning_fastai.unique())

Les valeurs uniques sont : ['no_reply' 'Fast_ai']


### Colonne use_ml_algorithms_gradient_boosting_machines :

In [16]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_ml_algorithms_gradient_boosting_machines_dic = {'use_ml_algorithms_gradient_boosting_machines' : {'Gradient Boosting Machines (xgboost, lightgbm, etc)' : 'Gradient Boosting Machines',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_ml_algorithms_gradient_boosting_machines_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_ml_algorithms_gradient_boosting_machines.unique())

Les valeurs uniques sont : ['no_reply' 'Gradient Boosting Machines']


### Colonne use_ml_algorithms_dense_neural_networks :

In [17]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_ml_algorithms_dense_neural_networks_dic = {'use_ml_algorithms_dense_neural_networks' : {'Dense Neural Networks (MLPs, etc)' : 'Dense Neural Networks',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_ml_algorithms_dense_neural_networks_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_ml_algorithms_dense_neural_networks.unique())

Les valeurs uniques sont : ['no_reply' 'Dense Neural Networks']


### Colonne use_ml_algorithms_transformer_networks :

In [18]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_ml_algorithms_transformer_networks_dic = {'use_ml_algorithms_transformer_networks' : {'Transformer Networks (BERT, gpt-3, etc)' : 'Transformer Networks ',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_ml_algorithms_transformer_networks_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_ml_algorithms_transformer_networks.unique())

Les valeurs uniques sont : ['Transformer Networks ' 'no_reply']


### Colonne use_computer_vision_methods_general_purpose_imagevideo_tools :

In [19]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_computer_vision_methods_general_purpose_imagevideo_tools_dic = {'use_computer_vision_methods_general_purpose_imagevideo_tools' : {'General purpose image/video tools (PIL, cv2, skimage, etc)' : 'General purpose image_video tools',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_computer_vision_methods_general_purpose_imagevideo_tools_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_computer_vision_methods_general_purpose_imagevideo_tools.unique())

Les valeurs uniques sont : ['no_reply' 'General purpose image_video tools']


### Colonne use_computer_vision_methods_image_segmentation_methods :

In [20]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_computer_vision_methods_image_segmentation_methods_dic = {'use_computer_vision_methods_image_segmentation_methods' : {'Image segmentation methods (U-Net, Mask R-CNN, etc)' : 'Image segmentation methods',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_computer_vision_methods_image_segmentation_methods_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_computer_vision_methods_image_segmentation_methods.unique())

Les valeurs uniques sont : ['Image segmentation methods' 'no_reply']


### Colonne use_computer_vision_methods_object_detection_methods :

In [21]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_computer_vision_methods_object_detection_methods_dic = {'use_computer_vision_methods_object_detection_methods' : {'Object detection methods (YOLOv3, RetinaNet, etc)' : 'Object detection methods',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_computer_vision_methods_object_detection_methods_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_computer_vision_methods_object_detection_methods.unique())

Les valeurs uniques sont : ['no_reply' 'Object detection methods']


### Colonne use_computer_vision_methods_image_classification_and_other_general_purpose_networks :

In [22]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_computer_vision_methods_image_classification_and_other_general_purpose_networks_dic = {'use_computer_vision_methods_image_classification_and_other_general_purpose_networks' : {'Image classification and other general purpose networks (VGG, Inception, ResNet, ResNeXt, NASNet, EfficientNet, etc)' : 'Image classification and other general purpose networks',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_computer_vision_methods_image_classification_and_other_general_purpose_networks_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_computer_vision_methods_image_classification_and_other_general_purpose_networks.unique())

Les valeurs uniques sont : ['Image classification and other general purpose networks' 'no_reply']


### Colonne use_computer_vision_methods_generative_networks :

In [23]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
use_computer_vision_methods_generative_networks_dic = {'use_computer_vision_methods_generative_networks' : {'Generative Networks (GAN, VAE, etc)' : 'Generative Networks',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(use_computer_vision_methods_generative_networks_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.use_computer_vision_methods_generative_networks.unique())

Les valeurs uniques sont : ['no_reply' 'Generative Networks']


### Colonne nlp_wordembeddings_vectors :

In [24]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
nlp_wordembeddings_vectors_dic = {'nlp_wordembeddings_vectors' : {'Word embeddings/vectors (GLoVe, fastText, word2vec)' : 'Word embeddings/vectors ',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(nlp_wordembeddings_vectors_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.nlp_wordembeddings_vectors.unique())

Les valeurs uniques sont : ['no_reply' 'Word embeddings/vectors ']


### Colonne nlp_encoder_decorder_models :

In [25]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
nlp_encoder_decorder_models_dic = {'nlp_encoder_decorder_models' : {'Encoder-decorder models (seq2seq, vanilla transformers)' : 'Encoder-decorder models',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(nlp_encoder_decorder_models_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.nlp_encoder_decorder_models.unique())

Les valeurs uniques sont : ['no_reply' 'Encoder-decorder models']


### Colonne nlp_contextualized_embeddings :

In [26]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
nlp_contextualized_embeddings_dic = {'nlp_contextualized_embeddings' : {'Contextualized embeddings (ELMo, CoVe)' : 'Contextualized embeddings',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(nlp_contextualized_embeddings_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.nlp_contextualized_embeddings.unique())

Les valeurs uniques sont : ['Contextualized embeddings' 'no_reply']


### Colonne nlp_transformerlanguage_models :

In [27]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
nlp_transformerlanguage_models_dic = {'nlp_transformerlanguage_models' : {'Transformer language models (GPT-3, BERT, XLnet, etc)'  : 'Transformer language models',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(nlp_transformerlanguage_models_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.nlp_transformerlanguage_models.unique())

Les valeurs uniques sont : ['Transformer language models' 'no_reply']


### Colonne these_activities_are_not_part_of_my_work :

In [28]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
these_activities_are_not_part_of_my_work_dic = {'these_activities_are_not_part_of_my_work' : {'None of these activities are an important part of my role at work' : 'Not part of my work',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(these_activities_are_not_part_of_my_work_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.these_activities_are_not_part_of_my_work.unique())

Les valeurs uniques sont : ['no_reply' 'Not part of my work']


### Colonne cloud_plateform_amazonwebservices :

In [29]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
cloud_plateform_amazonwebservices_dic = {'cloud_plateform_amazonwebservices' : {'Amazon Web Services (AWS)' : 'Amazon Web Services',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(cloud_plateform_amazonwebservices_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.cloud_plateform_amazonwebservices.unique())

Les valeurs uniques sont : ['Amazon Web Services' 'no_reply']


### Colonne cloud_plateform_googlecloudplatform :

In [30]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
cloud_plateform_googlecloudplatform_dic = {'cloud_plateform_googlecloudplatform' : {'Google Cloud Platform (GCP)' : 'Google Cloud Platform',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(cloud_plateform_googlecloudplatform_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.cloud_plateform_googlecloudplatform.unique())

Les valeurs uniques sont : ['Google Cloud Platform' 'no_reply']


### Colonne machinelearning_googlecloudaiplatform_googlecloudml :

In [31]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
machinelearning_googlecloudaiplatform_googlecloudml_dic = {'machinelearning_googlecloudaiplatform_googlecloudml' : {'Google Cloud AI Platform / Google Cloud ML Engine' : 'Google Cloud AI Platform_Google Cloud ML Engine',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(machinelearning_googlecloudaiplatform_googlecloudml_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.machinelearning_googlecloudaiplatform_googlecloudml.unique())

Les valeurs uniques sont : ['no_reply' 'Google Cloud AI Platform_Google Cloud ML Engine']


### Colonne ml_tool_automateddata :

In [32]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
ml_tool_automateddata_dic = {'ml_tool_automateddata' : {'Automated data augmentation (e.g. imgaug, albumentations)' : 'Automated data augmentation',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(ml_tool_automateddata_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.ml_tool_automateddata.unique())

Les valeurs uniques sont : ['no_reply' 'Automated data augmentation']


### Colonne ml_tool_automatedfeatureengineering :

In [33]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
ml_tool_automatedfeatureengineering_dic = {'ml_tool_automatedfeatureengineering' : {'Automated feature engineering/selection (e.g. tpot, boruta_py)' : 'Automated feature engineering_selection',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(ml_tool_automatedfeatureengineering_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.ml_tool_automatedfeatureengineering.unique())

Les valeurs uniques sont : ['no_reply' 'Automated feature engineering_selection']


### Colonne ml_tool_automatedmodelselection :

In [34]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
ml_tool_automatedmodelselection_dic = {'ml_tool_automatedmodelselection' : {'Automated model selection (e.g. auto-sklearn, xcessiv)' : 'Automated model selection',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(ml_tool_automatedmodelselection_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.ml_tool_automatedmodelselection.unique())

Les valeurs uniques sont : ['no_reply' 'Automated model selection']


### Colonne ml_tool_automatedmodelarchitecture :

In [35]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
ml_tool_automatedmodelarchitecture_dic = {'ml_tool_automatedmodelarchitecture' : {'Automated model architecture searches (e.g. darts, enas)' : 'Automated model architecture searches',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(ml_tool_automatedmodelarchitecture_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.ml_tool_automatedmodelarchitecture.unique())

Les valeurs uniques sont : ['no_reply' 'Automated model architecture searches']


### Colonne ml_tool_automatedhyperparameter :

In [36]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
ml_tool_automatedhyperparameter_dic = {'ml_tool_automatedhyperparameter' : {'Automated hyperparameter tuning (e.g. hyperopt, ray.tune, Vizier)' : 'Automated hyperparameter tuning',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(ml_tool_automatedhyperparameter_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.ml_tool_automatedhyperparameter.unique())

Les valeurs uniques sont : ['no_reply' 'Automated hyperparameter tuning']


### Colonne ml_tool_automationfullmlpipe :

In [37]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
ml_tool_automationfullmlpipe_dic = {'ml_tool_automationfullmlpipe' : {'Automation of full ML pipelines (e.g. Google AutoML, H20 Driverless AI)' : 'Automation of full ML pipelines',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(ml_tool_automationfullmlpipe_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.ml_tool_automationfullmlpipe.unique())

Les valeurs uniques sont : ['no_reply' 'Automation of full ML pipelines']


### Colonne primary_tool_to_analyze_data_text_response :

In [38]:
#Création d'un dictionnaire pour synthétiser les valeurs de la colonne :
primary_tool_to_analyze_data_text_response_dic = {'primary_tool_to_analyze_data_text_response' : {'Business intelligence software (Salesforce, Tableau, Spotfire, etc.)' :'Business intelligence software', 
 'Local development environments (RStudio, JupyterLab, etc.)': 'Local development environments',
 'Cloud-based data software & APIs (AWS, GCP, Azure, etc.)':'Cloud-based data software & APIs',
 'Basic statistical software (Microsoft Excel, Google Sheets, etc.)':'Basic statistical software',
 'Advanced statistical software (SPSS, SAS, etc.)' : 'Advanced statistical software',}}

#Mise à jour de la valeur de la colonne 
df_prep.replace(primary_tool_to_analyze_data_text_response_dic, inplace = True)

#Vérification
print('Les valeurs uniques sont :',df_prep.primary_tool_to_analyze_data_text_response.unique())

Les valeurs uniques sont : ['Business intelligence software' 'Local development environments'
 'no_reply' 'Cloud-based data software & APIs'
 'Basic statistical software' 'Advanced statistical software' 'Other']


## Machine learning 

In [39]:
#Import des librairies sklearn nécessaires à la standardisation et centralisation des données : 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [40]:
#Transformation des données catégorielles à l'aide de get_dummies :
df_prep_dm = pd.get_dummies(df_prep, dtype = np.int64)
display(df_prep_dm.head())
display(df_prep_dm.shape)

,diploma_bachelor,diploma_doctorate,diploma_master,diploma_no_information,diploma_no_superior_degree,diploma_professional_degree,programming_python_Python,programming_python_no_reply,programming_r_R,programming_r_no_reply,...,share_deploy_data_analysis_ml_dont_share_no_reply,share_deploy_data_analysis_ml_other_Other,share_deploy_data_analysis_ml_other_no_reply,primary_tool_to_analyze_data_text_response_Advanced statistical software,primary_tool_to_analyze_data_text_response_Basic statistical software,primary_tool_to_analyze_data_text_response_Business intelligence software,primary_tool_to_analyze_data_text_response_Cloud-based data software & APIs,primary_tool_to_analyze_data_text_response_Local development environments,primary_tool_to_analyze_data_text_response_Other,primary_tool_to_analyze_data_text_response_no_reply
0,0,0,1,0,0,0,1,0,1,0,...,1,0,1,0,0,1,0,0,0,0
1,0,0,1,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,1,0,1,0,...,1,0,1,0,0,0,0,0,0,1
3,0,1,0,0,0,0,1,0,0,1,...,1,0,1,0,0,0,1,0,0,0
4,1,0,0,0,0,0,1,0,0,1,...,1,0,1,0,1,0,0,0,0,0


(5670, 448)

In [41]:
features = df_prep_dm.copy()
display(features.head())

,diploma_bachelor,diploma_doctorate,diploma_master,diploma_no_information,diploma_no_superior_degree,diploma_professional_degree,programming_python_Python,programming_python_no_reply,programming_r_R,programming_r_no_reply,...,share_deploy_data_analysis_ml_dont_share_no_reply,share_deploy_data_analysis_ml_other_Other,share_deploy_data_analysis_ml_other_no_reply,primary_tool_to_analyze_data_text_response_Advanced statistical software,primary_tool_to_analyze_data_text_response_Basic statistical software,primary_tool_to_analyze_data_text_response_Business intelligence software,primary_tool_to_analyze_data_text_response_Cloud-based data software & APIs,primary_tool_to_analyze_data_text_response_Local development environments,primary_tool_to_analyze_data_text_response_Other,primary_tool_to_analyze_data_text_response_no_reply
0,0,0,1,0,0,0,1,0,1,0,...,1,0,1,0,0,1,0,0,0,0
1,0,0,1,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,1,0,1,0,...,1,0,1,0,0,0,0,0,0,1
3,0,1,0,0,0,0,1,0,0,1,...,1,0,1,0,0,0,1,0,0,0
4,1,0,0,0,0,0,1,0,0,1,...,1,0,1,0,1,0,0,0,0,0


In [42]:
#Création d'un ensemble d'entraînement et de test : 
X_train, X_test,y_train, y_test = train_test_split(features, target, test_size=0.2)

In [43]:
#Importation des librairies pour l'oversampling
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from sklearn.metrics import f1_score

### Oversampling

Oversampling aléatoire (RandomOverSampler) :  
*Formules :* RandomOverSampler().fit_resample(X_train, y_train)

SMOTE :  
*Formules :* X_sm, y_sm = smo.fit_resample(X_train, y_train)


#### Random OverSampler :

In [44]:
#Création d'une variable RandomOverSampler :
rOs = RandomOverSampler()
X_ro, y_ro = rOs.fit_resample(X_train, y_train)
print('Classes échantillon oversampled :', dict(pd.Series(y_ro).value_counts()))

Classes échantillon oversampled : {'Data Analyst': 2127, 'Data Scientist': 2127, 'Machine Learning Engineer': 2127, 'Data Engineer': 2127}


#### SMOTE

In [45]:
#Création d'une variable SMOTE :
smo = SMOTE()
X_sm, y_sm = smo.fit_resample(X_train, y_train)
print('Classes échantillon SMOTE :', dict(pd.Series(y_sm).value_counts()))

Classes échantillon SMOTE : {'Data Analyst': 2127, 'Data Scientist': 2127, 'Machine Learning Engineer': 2127, 'Data Engineer': 2127}


### Undersampling : 

Undersampling aléatoire (RandomUnderSampler) :  
*Formules :* RandomUnderSampler().fit_resample(X_train, y_train) 

Centroids :  
*Formules :* X_cc, y_cc = cc.fit_resample(X_train, y_train)


In [46]:
#Importation de la librairie pour l'undersampling
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids

#### Random Undersampling :

In [47]:
#Création d'une variable Random Undersampling :
rUs = RandomUnderSampler()
X_ru, y_ru = rUs.fit_resample(X_train, y_train)
print('Classes échantillon undersampled :', dict(pd.Series(y_ru).value_counts()))


Classes échantillon undersampled : {'Data Analyst': 363, 'Data Engineer': 363, 'Data Scientist': 363, 'Machine Learning Engineer': 363}


#### Centroids :

In [48]:
#Création d'une variable Centroids :
cc = ClusterCentroids()
X_cc, y_cc = cc.fit_resample(X_train, y_train)
print('Classes échantillon CC :', dict(pd.Series(y_cc).value_counts()))

Classes échantillon CC : {'Data Analyst': 363, 'Data Engineer': 363, 'Data Scientist': 363, 'Machine Learning Engineer': 363}


In [49]:
#importation des librairies pour le machine learning : 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import ensemble

In [50]:
#Importer la classe classification_report
from sklearn.metrics import classification_report

In [51]:
#Import du GridSearchCV : 
from sklearn.model_selection import GridSearchCV
import chime #il s'agit d'un module qui génère un son pour prévenir de la fin d'une tâche. Pour l'installer : pip install chime

In [52]:
#Création d'une variable par modèle simple de machine learning :  
dt_m = DecisionTreeClassifier()
clf = ensemble.RandomForestClassifier()

### Test sur X_ro, y_ro

In [53]:
#Score Decision Tree :
dt_m.fit(X_ro, y_ro)
print("score du DT : {}".format(dt_m.score(X_test, y_test)))

#Score Random Forest :
clf.fit(X_ro, y_ro)
print("score du RF : {}".format(clf.score(X_test, y_test)))

score du DT : 0.455026455026455
score du RF : 0.5890652557319224


### Test sur X_sm, y_sm

In [54]:
#Test avec ensemble oversamplé : 

#Score Decision Tree :
dt_m.fit(X_sm, y_sm)
print("score du DT : {}".format(dt_m.score(X_test, y_test)))

#Score Random Forest :
clf.fit(X_sm, y_sm)
print("score du RF : {}".format(clf.score(X_test, y_test)))

score du DT : 0.45943562610229277
score du RF : 0.582010582010582


### Test sur X_ru, y_ru

In [55]:
#Test avec ensemble undersamplé : 

#Score Decision Tree :
dt_m.fit(X_ru, y_ru)
print("score du DT : {}".format(dt_m.score(X_test, y_test)))

#Score Random Forest :
clf.fit(X_ru, y_ru)
print("score du RF : {}".format(clf.score(X_test, y_test)))

score du DT : 0.3800705467372134
score du RF : 0.5017636684303352


### Test sur X_cc, y_cc

In [56]:
#Test avec ensemble undersamplé : 

#Score Decision Tree :
dt_m.fit(X_cc, y_cc)
print("score du DT : {}".format(dt_m.score(X_test, y_test)))

#Score Random Forest :
clf.fit(X_cc, y_cc)
print("score du RF : {}".format(clf.score(X_test, y_test)))

score du DT : 0.22134038800705466
score du RF : 0.22486772486772486


### Premières observations :

Amélioration des scores comparés à la premières itérations, +10% avec le random forest.

### Abre de Décision : 

#### Test sur X_ro, y_ro : 

In [57]:
#Définition des paramètres : 
tree_param = {'criterion':['gini','entropy','log_loss'],'max_depth':np.arange(15,26)}

#Utilisation d'un gridsearchcv   
grid_dt = GridSearchCV(DecisionTreeClassifier(), tree_param)

#Fit du modèle sur X_train et y_train
grid_dt_fit = grid_dt.fit(X_ro, y_ro)

#Affichage du meilleur paramètre, estimator et score
print(pd.DataFrame(grid_dt.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec le DT est :",grid_dt_fit.best_score_)  
print("Le meilleur estimator obtenu avec le DT est :",grid_dt_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec le DT est :",grid_dt_fit.best_params_)

chime.error()

                                        params  mean_test_score  \
0       {'criterion': 'gini', 'max_depth': 15}         0.757765   
1       {'criterion': 'gini', 'max_depth': 16}         0.769872   
2       {'criterion': 'gini', 'max_depth': 17}         0.781155   
3       {'criterion': 'gini', 'max_depth': 18}         0.784565   
4       {'criterion': 'gini', 'max_depth': 19}         0.787738   
5       {'criterion': 'gini', 'max_depth': 20}         0.791851   
6       {'criterion': 'gini', 'max_depth': 21}         0.796084   
7       {'criterion': 'gini', 'max_depth': 22}         0.797964   
8       {'criterion': 'gini', 'max_depth': 23}         0.801608   
9       {'criterion': 'gini', 'max_depth': 24}         0.804311   
10      {'criterion': 'gini', 'max_depth': 25}         0.807132   
11   {'criterion': 'entropy', 'max_depth': 15}         0.774338   
12   {'criterion': 'entropy', 'max_depth': 16}         0.779392   
13   {'criterion': 'entropy', 'max_depth': 17}         0.78809

#### Test sur X_sm, y_sm : 

In [58]:
#Définition des paramètres : 
tree_param = {'criterion':['gini','entropy','log_loss'],'max_depth':np.arange(15,26)}

#Utilisation d'un gridsearchcv   
grid_dt = GridSearchCV(DecisionTreeClassifier(), tree_param)

#Fit du modèle sur X_train et y_train
grid_dt_fit = grid_dt.fit(X_sm, y_sm)

#Affichage du meilleur paramètre, estimator et score
print(pd.DataFrame(grid_dt.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec le DT est :",grid_dt_fit.best_score_)  
print("Le meilleur estimator obtenu avec le DT est :",grid_dt_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec le DT est :",grid_dt_fit.best_params_)

chime.error()

                                        params  mean_test_score  \
0       {'criterion': 'gini', 'max_depth': 15}         0.645997   
1       {'criterion': 'gini', 'max_depth': 16}         0.652697   
2       {'criterion': 'gini', 'max_depth': 17}         0.648701   
3       {'criterion': 'gini', 'max_depth': 18}         0.648349   
4       {'criterion': 'gini', 'max_depth': 19}         0.653520   
5       {'criterion': 'gini', 'max_depth': 20}         0.650465   
6       {'criterion': 'gini', 'max_depth': 21}         0.647879   
7       {'criterion': 'gini', 'max_depth': 22}         0.647057   
8       {'criterion': 'gini', 'max_depth': 23}         0.649172   
9       {'criterion': 'gini', 'max_depth': 24}         0.649525   
10      {'criterion': 'gini', 'max_depth': 25}         0.651407   
11   {'criterion': 'entropy', 'max_depth': 15}         0.642589   
12   {'criterion': 'entropy', 'max_depth': 16}         0.643882   
13   {'criterion': 'entropy', 'max_depth': 17}         0.64058

#### Test sur X_ru, y_ru : 

In [59]:
#Définition des paramètres : 
tree_param = {'criterion':['gini','entropy','log_loss'],'max_depth':np.arange(15,26)}

#Utilisation d'un gridsearchcv   
grid_dt = GridSearchCV(DecisionTreeClassifier(), tree_param)

#Fit du modèle sur X_train et y_train
grid_dt_fit = grid_dt.fit(X_ru, y_ru)

#Affichage du meilleur paramètre, estimator et score
print(pd.DataFrame(grid_dt.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec le DT est :",grid_dt_fit.best_score_)  
print("Le meilleur estimator obtenu avec le DT est :",grid_dt_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec le DT est :",grid_dt_fit.best_params_)

chime.error()

                                        params  mean_test_score  \
0       {'criterion': 'gini', 'max_depth': 15}         0.378140   
1       {'criterion': 'gini', 'max_depth': 16}         0.386420   
2       {'criterion': 'gini', 'max_depth': 17}         0.380223   
3       {'criterion': 'gini', 'max_depth': 18}         0.374710   
4       {'criterion': 'gini', 'max_depth': 19}         0.375393   
5       {'criterion': 'gini', 'max_depth': 20}         0.376751   
6       {'criterion': 'gini', 'max_depth': 21}         0.374698   
7       {'criterion': 'gini', 'max_depth': 22}         0.365738   
8       {'criterion': 'gini', 'max_depth': 23}         0.376753   
9       {'criterion': 'gini', 'max_depth': 24}         0.373314   
10      {'criterion': 'gini', 'max_depth': 25}         0.380206   
11   {'criterion': 'entropy', 'max_depth': 15}         0.398777   
12   {'criterion': 'entropy', 'max_depth': 16}         0.386373   
13   {'criterion': 'entropy', 'max_depth': 17}         0.39531

### Test sur X_cc, y_cc : 

In [60]:
#Définition des paramètres : 
tree_param = {'criterion':['gini','entropy','log_loss'],'max_depth':np.arange(15,26)}

#Utilisation d'un gridsearchcv   
grid_dt = GridSearchCV(DecisionTreeClassifier(), tree_param)

#Fit du modèle sur X_train et y_train
grid_dt_fit = grid_dt.fit(X_cc, y_cc)

#Affichage du meilleur paramètre, estimator et score
print(pd.DataFrame(grid_dt.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec le DT est :",grid_dt_fit.best_score_)  
print("Le meilleur estimator obtenu avec le DT est :",grid_dt_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec le DT est :",grid_dt_fit.best_params_)

chime.error()

                                        params  mean_test_score  \
0       {'criterion': 'gini', 'max_depth': 15}         0.465536   
1       {'criterion': 'gini', 'max_depth': 16}         0.473807   
2       {'criterion': 'gini', 'max_depth': 17}         0.475891   
3       {'criterion': 'gini', 'max_depth': 18}         0.461398   
4       {'criterion': 'gini', 'max_depth': 19}         0.472449   
5       {'criterion': 'gini', 'max_depth': 20}         0.475881   
6       {'criterion': 'gini', 'max_depth': 21}         0.478616   
7       {'criterion': 'gini', 'max_depth': 22}         0.471056   
8       {'criterion': 'gini', 'max_depth': 23}         0.471738   
9       {'criterion': 'gini', 'max_depth': 24}         0.482076   
10      {'criterion': 'gini', 'max_depth': 25}         0.460728   
11   {'criterion': 'entropy', 'max_depth': 15}         0.488236   
12   {'criterion': 'entropy', 'max_depth': 16}         0.482747   
13   {'criterion': 'entropy', 'max_depth': 17}         0.49653

### SCORE Arbre de décision :
*Première Itération* :  
**X_ro, y_ro : 78%**  
X_sm, y_sm : 56%  
X_ru, y_ru : 35%  
X_cc, y_cc : 49%

*Seconde Itération* :  
**X_ro, y_ro : 81%**  
X_sm, y_sm : 64%  
X_ru, y_ru : 39%  
X_cc, y_cc : 50%

#### Conclusion Decision Tree : 

Nous observons une progression de +3% dans le score du modèle **Decision Tree** en comparaison de la première itération grâce à l'ajout de nouveaux critères à X_train. 


### Matrice de confusion sur le meilleur résultat : 

In [61]:

#Utilisation des meilleurs paramètres avec le modèle Decision Tree :
dt_search = DecisionTreeClassifier(criterion = 'log_loss', max_depth = 24)
dt_search.fit(X_ro, y_ro)
y_pred = dt_search.predict(X_test)

#Calcul et affichage de classification_report
print( classification_report(y_test, y_pred) )

#Matrice de confusion
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

                           precision    recall  f1-score   support

             Data Analyst       0.46      0.49      0.48       287
            Data Engineer       0.06      0.07      0.06        74
           Data Scientist       0.58      0.54      0.56       549
Machine Learning Engineer       0.38      0.38      0.38       224

                 accuracy                           0.47      1134
                macro avg       0.37      0.37      0.37      1134
             weighted avg       0.48      0.47      0.47      1134



Classe prédite,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer
Classe réelle,,,,
Data Analyst,142,25,83,37
Data Engineer,27,5,31,11
Data Scientist,108,50,299,92
Machine Learning Engineer,29,10,101,84


#### Conclusion Matriuce de confusion du Decision Tree : 

On remarque une amélioration dans la classification des **Data Analyst** et **Data Scientist** par le modèle Decision Tree.

Il y a une légère amélioration dans la classification des **Machine Learning Engineer** et **Data Engineer** également.

L'un des éléments que nous avons remarqués lors de l'analyse de nos données est que la majorité des répondants partagent des compétences et utilisations de logiciels similaires. Ce qui empêche de facto le modèle de machine learning d'être en mesure de classifier avec précision les différents postes.

## Random Forest :

#### Test sur X_ro, y_ro :

In [62]:
#éfinition des paramètres  
rf = ensemble.RandomForestClassifier()  
param_grid_rf = [{'min_samples_split': [2, 31, 2],'max_features': ['sqrt', 'log2']}]

#Utilisation d'un gridsearchcv :  
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, refit=True)

#Fit le modèle sur X_train et y_train :  
grid_rf_fit = grid_rf.fit(X_ro, y_ro)

#Affichage du meilleur paramètre, estimator et score :  
print(pd.DataFrame(grid_rf.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec la RF est :",grid_rf_fit.best_score_)  
print("Le meilleur estimator obtenu avec la RF est :",grid_rf_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec la RF est :",grid_rf_fit.best_params_)

chime.error()

                                              params  mean_test_score  \
0   {'max_features': 'sqrt', 'min_samples_split': 2}         0.848731   
1  {'max_features': 'sqrt', 'min_samples_split': 31}         0.747062   
2   {'max_features': 'sqrt', 'min_samples_split': 2}         0.848613   
3   {'max_features': 'log2', 'min_samples_split': 2}         0.850729   
4  {'max_features': 'log2', 'min_samples_split': 31}         0.744711   
5   {'max_features': 'log2', 'min_samples_split': 2}         0.853550   

   std_test_score  
0        0.030424  
1        0.019544  
2        0.030763  
3        0.029256  
4        0.018471  
5        0.029346  
Le meilleur score obtenu avec la RF est : 0.8535496003761166
Le meilleur estimator obtenu avec la RF est : RandomForestClassifier(max_features='log2')
Le meilleur paramètre obtenu avec la RF est : {'max_features': 'log2', 'min_samples_split': 2}


#### Test sur X_sm, y_sm :

In [63]:
#Définition des paramètres  
rf = ensemble.RandomForestClassifier()  
param_grid_rf = [{'min_samples_split': [2, 31, 2],'max_features': ['sqrt', 'log2']}]

#Utilisation d'un gridsearchcv :  
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, refit=True)

#Fit le modèle sur X_train et y_train :  
grid_rf_fit = grid_rf.fit(X_sm, y_sm)

#Affichage du meilleur paramètre, estimator et score :  
print(pd.DataFrame(grid_rf.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec la RF est :",grid_rf_fit.best_score_)  
print("Le meilleur estimator obtenu avec la RF est :",grid_rf_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec la RF est :",grid_rf_fit.best_params_)

chime.error()

                                              params  mean_test_score  \
0   {'max_features': 'sqrt', 'min_samples_split': 2}         0.761871   
1  {'max_features': 'sqrt', 'min_samples_split': 31}         0.695581   
2   {'max_features': 'sqrt', 'min_samples_split': 2}         0.762106   
3   {'max_features': 'log2', 'min_samples_split': 2}         0.773625   
4  {'max_features': 'log2', 'min_samples_split': 31}         0.697226   
5   {'max_features': 'log2', 'min_samples_split': 2}         0.772097   

   std_test_score  
0        0.078786  
1        0.070752  
2        0.075876  
3        0.068724  
4        0.067403  
5        0.073131  
Le meilleur score obtenu avec la RF est : 0.7736248236953456
Le meilleur estimator obtenu avec la RF est : RandomForestClassifier(max_features='log2')
Le meilleur paramètre obtenu avec la RF est : {'max_features': 'log2', 'min_samples_split': 2}


#### Test sur X_ru, y_ru :

In [64]:
#Définition des paramètres  
rf = ensemble.RandomForestClassifier()  
param_grid_rf = [{'min_samples_split': [2, 31, 2],'max_features': ['sqrt', 'log2']}]

#Utilisation d'un gridsearchcv :  
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, refit=True)

#Fit le modèle sur X_train et y_train :  
grid_rf_fit = grid_rf.fit(X_ru, y_ru)

#Affichage du meilleur paramètre, estimator et score :  
print(pd.DataFrame(grid_rf.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec la RF est :",grid_rf_fit.best_score_)  
print("Le meilleur estimator obtenu avec la RF est :",grid_rf_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec la RF est :",grid_rf_fit.best_params_)

chime.error()

                                              params  mean_test_score  \
0   {'max_features': 'sqrt', 'min_samples_split': 2}         0.483471   
1  {'max_features': 'sqrt', 'min_samples_split': 31}         0.511019   
2   {'max_features': 'sqrt', 'min_samples_split': 2}         0.497245   
3   {'max_features': 'log2', 'min_samples_split': 2}         0.492424   
4  {'max_features': 'log2', 'min_samples_split': 31}         0.486915   
5   {'max_features': 'log2', 'min_samples_split': 2}         0.482094   

   std_test_score  
0        0.004463  
1        0.004245  
2        0.005924  
3        0.014545  
4        0.002577  
5        0.018659  
Le meilleur score obtenu avec la RF est : 0.5110192837465565
Le meilleur estimator obtenu avec la RF est : RandomForestClassifier(min_samples_split=31)
Le meilleur paramètre obtenu avec la RF est : {'max_features': 'sqrt', 'min_samples_split': 31}


#### Test sur X_cc, y_cc :

In [65]:
#Définition des paramètres  
rf = ensemble.RandomForestClassifier()  
param_grid_rf = [{'min_samples_split': [2, 31, 2],'max_features': ['sqrt', 'log2']}]

#Utilisation d'un gridsearchcv :  
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, refit=True)

#Fit le modèle sur X_train et y_train :  
grid_rf_fit = grid_rf.fit(X_cc, y_cc)

#Affichage du meilleur paramètre, estimator et score :  
print(pd.DataFrame(grid_rf.cv_results_)[['params', 'mean_test_score', 'std_test_score']])  
print("Le meilleur score obtenu avec la RF est :",grid_rf_fit.best_score_)  
print("Le meilleur estimator obtenu avec la RF est :",grid_rf_fit.best_estimator_)  
print("Le meilleur paramètre obtenu avec la RF est :",grid_rf_fit.best_params_)

chime.error()

                                              params  mean_test_score  \
0   {'max_features': 'sqrt', 'min_samples_split': 2}         0.623278   
1  {'max_features': 'sqrt', 'min_samples_split': 31}         0.619835   
2   {'max_features': 'sqrt', 'min_samples_split': 2}         0.617769   
3   {'max_features': 'log2', 'min_samples_split': 2}         0.618457   
4  {'max_features': 'log2', 'min_samples_split': 31}         0.618457   
5   {'max_features': 'log2', 'min_samples_split': 2}         0.615014   

   std_test_score  
0        0.038629  
1        0.050299  
2        0.039778  
3        0.045277  
4        0.045559  
5        0.054881  
Le meilleur score obtenu avec la RF est : 0.6232782369146005
Le meilleur estimator obtenu avec la RF est : RandomForestClassifier()
Le meilleur paramètre obtenu avec la RF est : {'max_features': 'sqrt', 'min_samples_split': 2}


### SCORE Random Forest :
*Première Itération :*  
**X_ro, y_ro : 80%**  
X_sm, y_sm : 68%  
X_ru, y_ru : 43%  
X_cc, y_cc : 56%

*Seconde Itération :*  
**X_ro, y_ro : 84%**  
X_sm, y_sm : 76%  
X_ru, y_ru : 50%  
X_cc, y_cc : 61%

#### Conclusion Random Forest : 

Nous observons une progression de +4% dans le score du modèle **Random Forest** en comparaison de la première itération grâce à l'ajout de nouveaux critères à X_train. 


### Matrice de confusion sur le meilleur résultat :

In [66]:
#Utilisation des meilleurs paramètres avec le modèle RF
rf = ensemble.RandomForestClassifier(max_features = 'sqrt', min_samples_split = 2)
rf.fit(X_ro, y_ro)
y_predict = rf.predict(X_test)

#Calcul et affichage de classification_report
print( classification_report(y_test, y_predict) )

#Matrice de confusion
pd.crosstab(y_test, y_predict, rownames=['Classe réelle'], colnames=['Classe prédite'])

                           precision    recall  f1-score   support

             Data Analyst       0.55      0.58      0.56       287
            Data Engineer       0.23      0.09      0.13        74
           Data Scientist       0.62      0.70      0.66       549
Machine Learning Engineer       0.52      0.42      0.47       224

                 accuracy                           0.57      1134
                macro avg       0.48      0.45      0.46      1134
             weighted avg       0.56      0.57      0.56      1134



Classe prédite,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer
Classe réelle,,,,
Data Analyst,166,11,92,18
Data Engineer,24,7,32,11
Data Scientist,98,9,384,58
Machine Learning Engineer,16,4,109,95


#### Conclusion Matriuce de confusion de la Random Forest : 

On remarque une amélioration dans la classification des **Data Analyst** et **Data Scientist** par le modèle *Random Forest*.  

Il y a une légère amélioration dans la classification des **Machine Learning Engineer** également. 

Néanmoins, le modèle reste toujours incapable de classifier correctement les personnes dont le poste est **Data Engineer**. La raison la plus probable est que le jeu de données à notre disposition ne dispose pas de suffisant de critères différenciant pour les **Data Engineer**.

L'un des éléments que nous avons remarqués lors de l'analyse de nos données est que la majorité des répondants partagent des compétences et utilisations de logiciels similaires. Ce qui empêche de facto le modèle de machine learning d'être en mesure de classifier avec précision les différents postes. 

## Conclusion Itération 2 : 

* Le travail de nettoyage et de standardisation des données a été relativement rapide malgré le grand nombre de colonnes analysées. 


* Cette 2eme itération, s'est focalisée sur les colonnes contenant uniquement des données de types **qualitatives** et sur l'utilisation des modèles de machine learning **Decision Tree** et **Random Forest**.  

* Nous avons constaté une amélioration dans le score des modèles de machine learning en comparaison de la première itération : 
    * Decision Tree passe de **78%** à **81%**
    * Random Forest passe de **80%** à **84%**
    
* Dans l'ensemble les classes sont mieux classifiées par les modèles, seule exception étant la classe : **Data Engineer**. 

* Ci-après nous avons essayé de trouver des explications à ce problème de classification :  
    * La raison la plus probable quant à ce problème de classification, vient de la nature de nos données. Pour rappel, il s'agit des réponses à une enquête réalisée par le site Kaggle sur les compétences et logiciels utilisés par les personnes qui travaillent dans la data. Il est possible que les questions posées soient trop *générales* et ne permettent pas de différencier suffisamment les répondants. 
    
    * Il est également possible que les personnes qui ont participé à l'enquête occupent des postes sur lesquels ils réalisent aussi bien des tâches propres aux : Data Analyst, Data Scientist, Data Engineer et Machine Learning Engineer.
    * Une autre possibilité est également un *enthousiasme* trop prononcé dans les réponses aux questionnaires, c'est-à-dire, indiquer l'utilisation de tels ou tels logiciels ou compétences, car ils en ont connaissance et/ou l'ont déjà utilisé, sans que cela ne reflète strictement leurs activités journalières. 
    
    * Enfin, il est bon de rappeler que Kaggle est une plateforme d'apprentissage et que les personnes qui visitent ce site le font avant tout pour se former. De ce fait, ce n'est pas *surprenant* d'avoir des Data Analyst, Data Scientist, Data Engineer et Machine Learning Engineer qui partagent des compétences et connaissances similaires. 


* Nous pouvons finalement, en conclure que nous avons sans doute atteint un niveau de performance satisfaisant avec notre modèle de machine learning. Pour améliorer encore plus la classification, il faudrait utiliser un nouveau jeu de données, qui cette fois-ci, s'appuierait par exemple : sur des données provenant directement d'offre d'emploi propre à chacun des métiers cible que nous avons étudiés. 

